In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99

Cloning into 'yolov5'...
remote: Enumerating objects: 14906, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 14906 (delta 11), reused 12 (delta 2), pack-reused 14880
Receiving objects: 100% (14906/14906), 13.92 MiB | 18.66 MiB/s, done.
Resolving deltas: 100% (10258/10258), done.
/content/yolov5
HEAD is now at fbe67e4 Fix `OMP_NUM_THREADS=1` for macOS (#8624)


In [ ]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

     |████████████████████████████████| 1.6 MB 42.6 MB/s 
Setup complete. Using torch 1.13.0+cu116 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [ ]:
#follow the link below to get your download code from from Roboflow
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="Far5TQ0Gqf6CkU9fHs95")
project = rf.workspace("uav-project-eia1l").project("uav_ir")
dataset = project.version(6).download("yolov5")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 1.0 MB/s 
     |████████████████████████████████| 67 kB 8.1 MB/s 
     |████████████████████████████████| 145 kB 57.8 MB/s 
     |████████████████████████████████| 178 kB 67.7 MB/s 
     |████████████████████████████████| 138 kB 77.1 MB/s 
     |████████████████████████████████| 54 kB 4.0 MB/s 
     |████████████████████████████████| 62 kB 2.0 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=4478b0a526db0fc2c20df781ba4fb8f8133c004ee7d737d4139dbfdf3ab9ebfe
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: certifi
    Found existing installation: certi

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Uav-11 in yolov5pytorch:: 100%|██████████| 14845/14845 [00:03<00:00, 4029.37it/s]


In [ ]:
%cd /content/yolov5
%cat {dataset.location}/data.yaml

/content/yolov5
names:
- Bird
- Drone
nc: 2
roboflow:
  license: CC BY 4.0
  project: uav-ce0zg
  url: https://universe.roboflow.com/detection-axsgy/uav-ce0zg/dataset/11
  version: 11
  workspace: detection-axsgy
test: ../test/images
train: Uav-11/train/images
val: Uav-11/valid/images


In [ ]:
# define number of classes based on YAML
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
#this is the model configuration we will use for our tutorial 
%cat //content/yolov5/models/yolov5s.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]]

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/yolov5/models/TFNet.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors: 3  # AutoAnchor evolves 3 anchors per P output layer
backbone: 
  [[-1, 1, Conv, [64, 3, 2]],  # 0-P1/2
   [-1, 3, BottleneckCSP, [64]],
   [-1, 1, Conv, [128, 5, 2]],  # 2-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 7, 2]],  # 4-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, SPP, [256, [5, 9, 13]]], # 5
  ]
head:
#Neck: PANet- feature fusion
  [[-1, 1, Conv, [128, 5, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 1], 1, Concat, [1]],  # cat backbone P1
   [-1, 3, BottleneckCSP, [128, False]],  # 13

   [-1, 1, Conv, [128, 5, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 3], 1, Concat, [1]],  # cat backbone P2
   [-1, 3, BottleneckCSP, [128, False]],  # 17 (P2/medium)

   [-1, 1, Conv, [128, 5, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 5], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [128, False]],  # 21 (P3/medium)
#Head: Yolo Layer
#Yolo Layer outputs detection results (class, score, location, size).
   [[13, 17, 21], 1, Detect, [nc, anchors]],  # Detect(P1, P2, P3)
  ]

In [ ]:
# train model for first time
%%time
%cd /content/yolov5/
!python train.py --img 320 --batch-size -1 --epochs 1000 --optimizer SGD --data {dataset.location}/data.yaml --cfg ./models/TFNet.yaml --hyp data/hyps/hyp.scratch-high.yaml --weights '' --project "/content/gdrive/MyDrive/Runs"  --name TFNet  --cache

In [ ]:
#Later, if it is stopped prematurely and I need to resume, I use
#!python train.py --img 320 --weights '' --project "/content/gdrive/MyDrive/Runs"  --name TFNet  --cache  --resume "/content/gdrive/MyDrive/Runs/TFNet/weights/last.pt"

In [ ]:
%cd /content/yolov5/
!python detect.py --weights /content/gdrive/MyDrive/Runs/TFNet/weights/best.pt --img 320 --conf 0.3 --source /content/yolov5/Uav-11/valid/images 

In [ ]:
!python val.py --task study --weights /content/gdrive/MyDrive/Runs/TFNet/weights/best.pt --data {dataset.location}/data.yaml --iou 0.5

In [ ]:
!python val.py --task speed --weights /content/gdrive/MyDrive/Runs/TFNet/weights/best.pt --data {dataset.location}/data.yaml --iou 0.5

In [ ]:
!python val.py --img 320 --weights /content/gdrive/MyDrive/Runs/TFNet/weights/best.pt --data {dataset.location}/data.yaml --iou 0.5 

In [ ]:
#%cd /content/yolov5/
#!python detect.py --weights runs/train/MFNet_small2_results/weights/best.pt --img 320 --conf 0.3 --source /content/yolov5/Uav-7/valid/images 

In [ ]:
#### Visulize each layers feature map
#######Run this cell to know the whats inside every lare feature
#### stop it ur self after one image is done
## Print whole folder feature maps and save them
#!python detect.py --weights /content/yolov5/mfnets.pt --img 416 --conf 0.4 --source /content/yolov5/Uav-7/test/images --visualize
####### Only single image
#!python detect.py --weights /content/yolov5/mfnets.pt --img 416 --conf 0.4 --source /content/yolov5/Uav-7/test/images/0094_jpg.rf.09856e96b7f9f77c2bc502452ec20d77.jpg --visualize
                                               

In [ ]:
#!python val.py --task study --weights runs/train/MFNet_small2_results2/weights/best.pt --data {dataset.location}/data.yaml --iou 0.5

In [ ]:
#!python val.py --task speed --weights runs/train/MFNet_small2_results2/weights/best.pt--data {dataset.location}/data.yaml --iou 0.5

In [ ]:
#!python val.py --img 320 --weights runs/train/MFNet_small2_results2/weights/best.pt --data {dataset.location}/data.yaml --iou 0.5 

In [ ]:
!zip -r /content/TFNet-nightvision-SGD.zip /content/gdrive/MyDrive/Runs/
!zip -r /content/TFNet-nightvision-Adam.zip /content/yolov5/runs/
from google.colab import files
files.download("/content/TFNet-nightvision-Adam.zip")